In [1]:
import re
import nltk
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('stopwords')
import numpy as np
import pandas as pd
import heapq
import numpy as np
from collections import Counter

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
podcasts_df = pd.read_csv('/content/drive/MyDrive/bookfinal/book_final_data.csv')

In [3]:
podcasts_df.head()

,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Language,Category,city,state,country
0,11676,34,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],unknown,unknown,unknown
1,116866,34,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,",",unknown
2,110912,36,0374157065,10,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an o...",en,['Medical'],milpitas,california,usa
3,197659,49,0374157065,9,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an o...",en,['Medical'],indiana,pennsylvania,usa
4,11676,34,0399135782,9,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,A Chinese immigrant who is convinced she is dy...,en,['Fiction'],unknown,unknown,unknown


In [4]:
podcasts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153428 entries, 0 to 153427
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              153428 non-null  int64  
 1   age                  153428 non-null  int64  
 2   isbn                 153428 non-null  object 
 3   rating               153428 non-null  int64  
 4   book_title           153428 non-null  object 
 5   book_author          153428 non-null  object 
 6   year_of_publication  153428 non-null  float64
 7   publisher            153428 non-null  object 
 8   Summary              153428 non-null  object 
 9   Language             153428 non-null  object 
 10  Category             153428 non-null  object 
 11  city                 153428 non-null  object 
 12  state                153428 non-null  object 
 13  country              153428 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 16.4+ MB


In [5]:

podcasts_df.dropna(inplace=True)

In [6]:
podcasts_df['book_title'].duplicated().sum()

73845

In [7]:
podcasts_df.drop_duplicates(subset ="book_title",
                     keep = 'last', inplace = True)

In [8]:
podcasts_df['book_title'].duplicated().sum()

0

In [9]:
podcasts_df.head(2)

,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Summary,Language,Category,city,state,country
1,116866,34,0002005018,9,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,",",unknown
3,197659,49,0374157065,9,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,"Describes the great flu epidemic of 1918, an o...",en,['Medical'],indiana,pennsylvania,usa


In [10]:
podcasts_df=podcasts_df.iloc[0:9000].reset_index()

In [11]:
podcasts_df['text'] = podcasts_df[['book_title', 'book_author', 'publisher', 'Summary','Category']].apply(lambda x: " ".join(x), axis=1)
podcasts_df=podcasts_df[['book_title','text']]
podcasts_df.head()

,book_title,text
0,Clara Callan,Clara Callan Richard Bruce Wright HarperFlamin...
1,Flu: The Story of the Great Influenza Pandemic...,Flu: The Story of the Great Influenza Pandemic...
2,What If?: The World's Foremost Military Histor...,What If?: The World's Foremost Military Histor...
3,PLEADING GUILTY,PLEADING GUILTY Scott Turow Audioworks 9 9
4,Beloved (Plume Contemporary Fiction),Beloved (Plume Contemporary Fiction) Toni Morr...


In [12]:
podcasts_df['text'].sample(1)

889    Die Nadel Ken Follett Gustav Lubbe Verlag GmbH...
Name: text, dtype: object

In [13]:
# create list of stop words
stop = stopwords.words('english')

# remove non-alphanumeric, non-space
stop = [re.sub(r'([^\s\w]|_)+', '', x) for x in stop]

def topKFrequent(tokenized_text, k): 
   
    count = Counter(tokenized_text)   
    
    return heapq.nlargest(k, count.keys(), key=count.get)

def remove_stop(text, stop):
    custom_stop = stop
#     top5 = topKFrequent(text, 5)
#     custom_stop = custom_stop + top5
    
    new_text = []
    for word in text:
        if word not in custom_stop:
            new_text.append(word)
    return new_text

# create tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# create stemmer
p_stemmer = PorterStemmer()
l_stemmer = WordNetLemmatizer() 


def stem_list(text, p_stemmer):
    new_list = []
    for word in text:
        new_list.append(p_stemmer.stem(word))
    return new_list

def lem_list(text, l_stemmer):
    new_list = []
    for word in text:
        new_list.append(l_stemmer.lemmatize(word))
    return new_list

def preprocess_text(text):
    # remove mixed alphanumeric
    text = re.sub(r"""(?x) # verbose regex
                            \b    # Start of word
                            (?=   # Look ahead to ensure that this word contains...
                             \w*  # (after any number of alphanumeric characters)
                             \d   # ...at least one digit.
                            )     # End of lookahead
                            \w+   # Match the alphanumeric word
                            \s*   # Match any following whitespace""", 
                             "", text)
    
    # remove urls (will check and remove http and www later)
    text = re.sub(r'\s([\S]*.com[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.org[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.net[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.edu[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.gov[\S]*)\b', '', text)
    
    # remove non-alphanumeric, non-space
    text = re.sub(r'([^\s\w]|_)+', '', text)
    
    # tokenize text
    text = tokenizer.tokenize(text.lower())
    
    # remove stop words
    text = remove_stop(text, stop)
    
    # stem
    text = lem_list(text, l_stemmer)
    
    # remove instances of http or www
    new_text_list = []
    for word in text:
        if re.search(r'http', word):
            continue
        if re.search(r'www', word):
            continue
        new_text_list.append(word)
    
    new_text = ' '.join(new_text_list)
    
    return new_text

In [14]:
podcasts_df['text'] = podcasts_df['text'].map(preprocess_text)
podcasts_df = podcasts_df[podcasts_df.text != '']

In [15]:
podcasts_df

,book_title,text
0,Clara Callan,clara callan richard bruce wright harperflamin...
1,Flu: The Story of the Great Influenza Pandemic...,flu story great influenza pandemic search viru...
2,What If?: The World's Foremost Military Histor...,world foremost military historian imagine migh...
3,PLEADING GUILTY,pleading guilty scott turow audioworks
4,Beloved (Plume Contemporary Fiction),beloved plume contemporary fiction toni morris...
...,...,...
8995,Flame Of Recca (Flame Of Recca),flame recca flame recca nobuyuki anzai viz com...
8996,Love Hina (Book 3),love hina book ken akamatsu tokyopop
8997,"Real Bout High School, Book 3",real bout high school book reiji saiga tokyopop
8998,"Real Bout High School, Book 4",real bout high school book reiji saiga tokyopo...


In [16]:
def get_recommendations(title, sim_matrix):
    index=podcasts_df[podcasts_df['book_title']==title].index[0]
    distances=sorted(list(enumerate(sim_matrix[index])),reverse=True,key=lambda x:x[1])
    for i in distances[1:6]:
      print(podcasts_df.iloc[i[0]].book_title)                 


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer(max_features=5000,stop_words='english')
cv_matrix = cv.fit_transform(podcasts_df["text"]).toarray()
cv_cosine_sim = cosine_similarity(cv_matrix)

In [18]:
get_recommendations('Beloved (Plume Contemporary Fiction)', cv_cosine_sim)


How the Garcia Girls Lost Their Accents (Plume Contemporary Fiction)
A Wild Sheep Chase: A Novel (Plume Contemporary Fiction)
Black Water (Plume Contemporary Fiction)
Story of O
Chang and Eng: A Novel


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf_matrix = tf.fit_transform(podcasts_df["text"])
tf_cosine_sim = cosine_similarity(tf_matrix)

In [20]:
get_recommendations('Beloved (Plume Contemporary Fiction)', tf_cosine_sim)


Beloved : Gift Edition
How the Garcia Girls Lost Their Accents (Plume Contemporary Fiction)
Song of Solomon (Oprah's Book Club (Paperback))
A Wild Sheep Chase: A Novel (Plume Contemporary Fiction)
Wilderness : The Lost Writings of Jim Morrison (Morrison, Jim, Lost Writings of Jim Morrison (Vintage Books (Firm)), V. 1.)


In [21]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [22]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [23]:

text_list = list(podcasts_df.text)
tokenized_text = [tokenizer.tokenize(i) for i in text_list]

In [24]:
w2v_model = Word2Vec(tokenized_text, sg=1)


In [25]:
nltk.download('punkt')
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v_model)
mean_embedded = mean_embedding_vectorizer.fit_transform(podcasts_df['text'])
w2v_cosine_sim = cosine_similarity(mean_embedded)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [26]:
get_recommendations('Beloved (Plume Contemporary Fiction)', w2v_cosine_sim)


The Notebooks of Don Rigoberto
Vox: A Novel
Oracles and Miracles
Joke
Midnight's Children


In [27]:
from scipy.sparse.linalg import svds
from sklearn.decomposition import TruncatedSVD

In [28]:
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX

def smooth_inverse_frequency(sent, a=0.001, word2vec_model=w2v_model):
    word_counter = {}
    sentences = []
    total_count = 0
    no_of_sentences = 0
    
    for s in sent:
        for w in s:
            if w in word_counter:
                word_counter[w] = word_counter[w] + 1
            else:
                word_counter[w] = 1
        total_count = total_count + len(s)
        no_of_sentences = no_of_sentences + 1
    
    sents_emd = []
    for s in sent:
        sent_emd = []
        for word in s:
            if word in word2vec_model:
                emd = (a/(a + (word_counter[word]/total_count)))*word2vec_model[word]
                sent_emd.append(emd)
        sum_ = np.array(sent_emd).sum(axis=0)
        sentence_emd = sum_/float(no_of_sentences)
        sents_emd.append(sentence_emd)
        
    new_sents_emb = remove_first_principal_component(np.array(sents_emd))
    return new_sents_emb

In [29]:
sif_text_emd = smooth_inverse_frequency(text_list)
sif_cosine_sim = cosine_similarity(sif_text_emd)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [30]:
get_recommendations('Beloved (Plume Contemporary Fiction)', sif_cosine_sim)


Man, Woman, and the Meaning of Love: God's Plan for Love, Marriage, Intimacy, and the Family
Flatland: A Romance of Many Dimensions (Dover Thrift Editions)
Confucio nel computer: Memoria accidentale del futuro
The Nightspinners : A Novel
Three Lives: Stories of the Good Anna, Melanctha and the Gentle Lena (Dover Thrift Editions)
